# Association Rule Learning using the Apriori Algorithm

| Key              | Value                                                                                                                                                                                                                                                                                                        |
|:-----------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Course Codes** | BBT 4206 and BFS 4102                                                                                                                                                                                                                                                                                        |
| **Course Names** | BBT 4206: Business Intelligence II (Week 1-3 of 13) and<br/>BFS 4102: Advanced Business Data Analytics (Week 4-6 of 13)                                                                                                                                                                                      |
| **Semester**     | August to November 2025                                                                                                                                                                                                                                                                                      |
| **Lecturer**     | Allan Omondi                                                                                                                                                                                                                                                                                                 |
| **Contact**      | aomondi@strathmore.edu                                                                                                                                                                                                                                                                                       |
| **Note**         | The lecture contains both theory and practice.<br/>This notebook forms part of the practice.<br/>It is intended for educational purposes only.<br/>Recommended citation: [BibTex](https://raw.githubusercontent.com/course-files/ClusteringandAssociationRuleMining/refs/heads/main/RecommendedCitation.bib) |

**Business context:** A supermarket chain seeks to uncover frequent item combinations from historical transactions to improve product placement, plan promotions, and increase cross-selling.

**Dataset:** The **"groceries"** dataset by **Hahsler et al. (2011)** contains 9,835 market basket transactions. Each row in the dataset represents items in a customer's shopping cart.

## Step 1: Import the necessary libraries

**Purpose**: This chunk imports all the necessary libraries for data analysis, machine learning, and visualization.

1. **For File and system operations [urllib3](https://urllib3.readthedocs.io/en/stable/)**
    - `urllib.request` is used for opening and downloading data from URLs.
    - `from pathlib import Path` is used to handle file paths in a platform-independent way.
    - `os` provides functions for interacting with the operating system, such as file and directory management.

2. **For data manipulation - [pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html):**
    - `pandas as pd`: For loading the dataset, creating and managing DataFrames, data manipulation and analysis using DataFrames

3. **For data preprocessing and transformation - [mlxtend](https://rasbt.github.io/mlxtend/)**
    - `TransactionEncoder`: Converts transaction data into a binary matrix format

4. **For association rule learning - [mlxtend](https://rasbt.github.io/mlxtend/)**
    - `apriori`: Implements the Apriori algorithm for finding frequent itemsets
    - `association_rules`: Generates rules from frequent itemsets

5. **For data visualization - [matplotlib](https://matplotlib.org/stable/gallery/index.html) and [seaborn](https://seaborn.pydata.org/examples/index.html)**
    - `matplotlib.pyplot as plt`: For basic plotting functionality
    - `seaborn as sns`: For enhanced statistical visualizations

6. **For suppressing warnings - [warnings](https://docs.python.org/3/library/warnings.html)**
    - `warnings`: Controls warning messages
    - `warnings.filterwarnings('ignore')`: Suppresses warning messages for cleaner output
    - Used to suppress warnings that may arise during the execution of the code. Even though it is not necessary for the code to run, it helps in keeping the output clean and focused on the results.

In [ ]:
# For file and system operations
import urllib.request
from pathlib import Path
import os

# For data manipulation
import pandas as pd

# For data preprocessing and transformation
from mlxtend.preprocessing import TransactionEncoder

# For Association Rule Learning
from mlxtend.frequent_patterns import apriori, association_rules

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For suppressing warnings
import warnings
warnings.filterwarnings('ignore')

## Step 2: Load the data

1. **Data Loading Process**
    - Uses a context manager (`with` statement) for proper resource handling
    - Steps:
        1. Opens URL connection
        2. Reads binary content
        3. Decodes from bytes to UTF-8 string
        4. Splits into lines and processes each line

2. **Data Processing**
    - List comprehension transforms raw data into a structured format
    - Each transaction is processed by:
        - `strip()`: Removes whitespace
        - `split(',')`: Creates a list of items from comma-separated values

3. **Output**
    - Prints the total number of transactions
    - Shows the first three transactions as a sample


In [ ]:
dataset_path = './data/groceries.csv'
url = 'https://github.com/course-files/ClusteringandAssociationRuleMining/raw/refs/heads/main/data/groceries.csv'

if not os.path.exists(dataset_path):
    print("Downloading dataset...")
    if not os.path.exists('./data'):
        os.makedirs('./data')
    urllib.request.urlretrieve(url, dataset_path)
    print("✅ Dataset downloaded")
else:
    print("✅ Dataset already exists locally")

path = Path(dataset_path)
content = path.read_text(encoding='utf-8')

transactions = []   # Start with an empty list

# Go through each line in the text
for line in content.splitlines():
    # Remove spaces and check if the line is not empty
    if line.strip():
        items = []  # Start a new shopping cart (list of items)

        # Go through each item in this line
        for item in line.split(","):
            # Remove spaces and ignore empty items
            if item.strip():
                items.append(item.strip())

        # Add this cleaned shopping cart to the transactions list
        transactions.append(items)

print(f"Total number of transactions: {len(transactions)}")
print("\nFirst three transactions:")
transactions[:3]

## Step 3: Convert the transaction list into a one-hot encoded DataFrame

**Purpose:**

The purpose of this chunk is to format the transaction data correctly. This is essential for:
- Running the Apriori algorithm
- Computing association rules
- Analyzing item frequencies and relationships

1. **Transaction Encoder Initialization**
    - Creates a new `TransactionEncoder` object called `encoder`
    - Purpose: To convert transaction lists into a binary matrix format

2. **Fit and Transform Process**
    - `encoder.fit(transactions)`:
        - Learns all unique items across all transactions
        - Creates a mapping of items to columns

    - `transform(transactions)`:
        - Converts transactions into a binary matrix
        - Each row represents one transaction
        - Each column represents one item
        - Values: True/False indicating item presence

3. **DataFrame Creation**
    - Converts the binary matrix into a `pandas` DataFrame
    - Uses `encoder.columns_` to retrieve the column names
    - Each column name is a unique item
    - Each row shows items present (True) or absent (False)

4. **Data Preview**
    - `transaction_data.head()`: Shows first five rows of transformed data

**Example:**
If the original transactions were:
```
Transaction 1: ["milk", "bread"]
Transaction 2: ["bread", "butter"]
```
The transformed data would look like this:
```
   milk  bread  butter
0  True   True  False
1  False  True   True
```

In [ ]:
encoder = TransactionEncoder()
onehot = encoder.fit(transactions).transform(transactions)
transaction_data = pd.DataFrame(onehot, columns=encoder.columns_)
transaction_data.head()

## Step 4: Generate frequent `itemsets` using the Apriori algorithm

1. **Apriori Algorithm Application**
    - Function: `apriori()`
    - Parameters:
        - `transaction_data`: Binary encoded transaction matrix
        - `min_support=0.02`: Minimum support threshold (2%)
        - `use_colnames=True`: Use item names instead of indices

2. **Support Threshold**
    - Support(itemset X) = (number of transactions containing itemset X) / (total transactions)
    - 0.02 means itemset X must appear in at least 2% of transactions
    - Helps filter out rare combinations

3. **Result Generation**
    - Creates a DataFrame containing:
        - `itemsets`: Combinations of items
        - `support`: Frequency of occurrence

4. **Result Processing**
    - `sort_values(by='support', ascending=False)`: Orders by support value (highest first)
    - `head(10)`: Shows top 10 most frequent itemsets

In [ ]:
frequent_itemsets = apriori(transaction_data, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head(10)

## Step 5: Generate and display the association rules

1. **Rules Generation**
    - Function: `association_rules()`
    - Input: `frequent_itemsets` from Apriori algorithm
    - Parameters:
        - `metric='lift'`: A measure of how much more often items in the antecedent and the items in the consequent appear together in transactions compared to what would be expected if they were statistically independent.
        - `min_threshold=1.0`: Minimum lift value to include

2. **Rules and Key Metrics**
    - `antecedents`: "If" part of the rule (items in a basket)
    - `consequents`: "Then" part of the rule (likely additional items)
    - `support`: Frequency of items appearing together
    - `confidence`: Probability of consequent given antecedent
    - `lift`: Ratio of observed support to expected support

3. **Sorting and Display**
    - Sorts rules by confidence (highest first)
    - Shows the top 10 strongest associations
    - Displays most relevant columns for analysis

In [ ]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)

### Display the top 10 rules sorted by confidence

In [ ]:
rules = rules.sort_values(by='confidence', ascending=False)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

### Display the top 10 rules sorted by lift and then confidence

In [ ]:
rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

### Filter the rules to show only those with high confidence and high lift

In [ ]:
strong_rules = rules[
    (rules['confidence'] >= 0.2) &
    (rules['lift'] >= 2.0)
].sort_values(by='confidence', ascending=False)
strong_rules.head(15)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

## Step 6: Remove duplicate and redundant rules

In [ ]:
def remove_duplicates(rules_df):
    rules_df = rules_df.sort_values(by=['lift', 'confidence'], ascending=[False, False]).reset_index(drop=True)
    unique_rules = []

    for i, row_i in rules_df.iterrows():
        is_redundant = False
        for j, row_j in enumerate(unique_rules):
            if row_i['consequents'] == row_j['consequents'] and row_i['antecedents'].issubset(row_j['antecedents']):
                is_redundant = True
                break

        if not is_redundant:
            unique_rules.append(row_i)

    return pd.DataFrame(unique_rules)

# Apply the function to the strong rules
nonredundant_rules = remove_duplicates(strong_rules)
nonredundant_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [ ]:
rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

In [ ]:
# Eliminate bidirectional redundancy in rules
def remove_bidirectional_redundancy(rules_df):
    # Create a set to track unique antecedent-consequent pairs
    seen_rules = set()
    filtered_rules = []  # Store the final filtered rules

    for _, row in rules_df.iterrows():
        # Combine antecedents and consequents into a frozenset so the order doesn't matter
        rule_pair = frozenset([frozenset(row['antecedents']), frozenset(row['consequents'])])

        # Only keep the rule if it hasn't already been seen
        if rule_pair not in seen_rules:
            seen_rules.add(rule_pair)
            filtered_rules.append(row)

    return pd.DataFrame(filtered_rules)

# Apply the function to the strong rules
cleaned_rules = remove_bidirectional_redundancy(nonredundant_rules)
cleaned_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

**List of Rules**

In [ ]:
cleaned_rules = cleaned_rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
cleaned_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

## Step 7: Save the rules as a CSV file

In [ ]:
# Define the output path
output_path = './rule/top_rules_7b.csv'

# Ensure the data directory exists
if not os.path.exists('./rule'):
    os.makedirs('./rule')

# Save the top rules as a CSV file
cleaned_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_csv(output_path, index=False)
print(f"\n✅ Top rules saved to {output_path}")

# Provide a download link if running in Google Colab
try:
    from google.colab import files
    files.download(output_path)
except ImportError:
    print("❌ Not running in Google Colab, skipped rule download link.")

## Step 8: Visualize rules

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(data=rules, x='support', y='confidence', size='lift', hue='lift', palette='viridis', sizes=(30, 300))
plt.title('Support vs Confidence of Association Rules')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.grid(True)
plt.legend(title='Lift', bbox_to_anchor=(1.15, 1), loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(data=rules, x='lift', y='confidence', size='lift', hue='lift', palette='viridis', sizes=(30, 300))
plt.title('Lift vs Confidence of Association Rules')
plt.xlabel('Lift')
plt.ylabel('Confidence')
plt.grid(True)
plt.legend(title='Lift', bbox_to_anchor=(1.15, 1), loc='upper right')
plt.tight_layout()
plt.show()

# References
Hahsler, M., Chelluboina, S., Hornik, K., & Buchta, C. (2011). The arules R-Package Ecosystem: Analyzing Interesting Patterns from Large Transaction Datasets. Journal of Machine Learning Research, 12, 1977–1981.


# Lab Submission

## Student Details

**Name of the team on GitHub Classroom:**

**Team Member Contributions**

**Member 1**

| **Details**                                                                                        | **Comment** |
|:---------------------------------------------------------------------------------------------------|:------------|
| **Student ID**                                                                                     |             |
| **Name**                                                                                           |             |
| **What part of the lab did you personally contribute to,** <br>**and what did you learn from it?** |             |

**Member 2**

| **Details**                                                                                        | **Comment** |
|:---------------------------------------------------------------------------------------------------|:------------|
| **Student ID**                                                                                     |             |
| **Name**                                                                                           |             |
| **What part of the lab did you personally contribute to,** <br>**and what did you learn from it?** |             |

**Member 3**

| **Details**                                                                                        | **Comment** |
|:---------------------------------------------------------------------------------------------------|:------------|
| **Student ID**                                                                                     |             |
| **Name**                                                                                           |             |
| **What part of the lab did you personally contribute to,** <br>**and what did you learn from it?** |             |

**Member 4**

| **Details**                                                                                        | **Comment** |
|:---------------------------------------------------------------------------------------------------|:------------|
| **Student ID**                                                                                     |             |
| **Name**                                                                                           |             |
| **What part of the lab did you personally contribute to,** <br>**and what did you learn from it?** |             |

**Member 5**

| **Details**                                                                                        | **Comment** |
|:---------------------------------------------------------------------------------------------------|:------------|
| **Student ID**                                                                                     |             |
| **Name**                                                                                           |             |
| **What part of the lab did you personally contribute to,** <br>**and what did you learn from it?** |             |

## Instructions

You are working as a data analyst for a retail store. Using the association rules derived from the dataset (`cleaned_rules`), you are required to provide business recommendations and design a simple recommender function.

### Part 1: Business Analysis (Critical Thinking)

- Examine the association rules in `cleaned_rules`.
- Present a business analysis report that addresses:
  - Which rules are useful and actionable for the business? (e.g., cross-promotions, store layout, bundling strategies).
  - Which rules are misleading or not useful despite appearing strong? (Consider support, confidence, and lift).
  - Recommend two specific business actions the company should implement based on your findings.

### Part 2: Recommender System (Programming)

**Baseline (Required)**
- Create a function called `dynamic_recommender_baseline(cart, rules_df)` that:
  - Accepts a list of products in the client’s cart as the antecedent.
  - Searches `cleaned_rules` for matching antecedents.
  - Returns the consequent(s) as the recommendations.

**Intermediate (Recommended)**
- Extend your function by creating another one named `dynamic_recommender_intermediate(cart, rules_df)` that is extended to:
  - Handle multiple items in the shopping cart.
  - If no rule matches, return "No recommendation available."
  - Rank recommendations using confidence (highest first).

**Advanced (Optional Challenge)**
- Enhance your recommender by creating another function named `dynamic_recommender_advanced(cart, rules_df)` that is extended to:
  - Allowing the user to request top-N recommendations.
  - Incorporating lift to break ties between recommendations.
  - Comparing your recommender’s output to a naïve baseline (e.g., always recommend top-selling products).
  - Briefly discuss: When would association rule recommenders fail in real businesses?

### Deliverables

1. Business Analysis Report.
2. Python implementation of the `dynamic_recommender` function. State whether you completed the Baseline, Intermediate, or Advanced version.
3. Sample runs of your function showing different shopping carts and outputs.

### Grading Approach

- Baseline = Pass (implementation works, basic analysis) >= 60%
- Intermediate = Merit (handles edge cases, thoughtful analysis) 75–85%
- Advanced = Distinction (robust function, strategic business insights) >= 86%

**Note 1:** The real challenge is not building the function—it is deciding which rules actually matter for the business. A good analyst filters the noise, questions misleading correlations, and translates data into profitable actions.

**Note 2:** The sophistication of the recommender is not measured by code complexity, but by how well it handles real-world edge cases—like sparse data, missing rules, and ambiguous ties.